In [1]:
from pythtb import Lattice, Mesh, TBModel, WFArray
import numpy as np

In [2]:
lattice = Lattice(
    lat_vecs=[[1.0]], orb_vecs=[[0.0], [1 / 3], [2 / 3]], periodic_dirs=[0]
)

In [3]:
mesh = Mesh(["k"])

In [4]:
mesh.build_grid([20])

In [5]:
wfa = WFArray(lattice=lattice, mesh=mesh)

In [6]:
t = -1.3
delta = 2.0

model = TBModel(lattice=lattice)

# nearest-neighbour hoppings (last hop wraps to the next cell)
model.set_hop(t, 0, 1, [0])
model.set_hop(t, 1, 2, [0])
model.set_hop(t, 2, 0, [1])

# Per-orbital onsite as lambdas of lmbda
onsite = [
    lambda lmbda: delta * -np.cos(2 * np.pi * (lmbda - 0 / 3)),
    lambda lmbda: delta * -np.cos(2 * np.pi * (lmbda - 1 / 3)),
    lambda lmbda: delta * -np.cos(2 * np.pi * (lmbda - 2 / 3)),
]

model.set_onsite(onsite)

In [7]:
model_fixed = model.with_parameters(lmbda=0.25)
wfa.solve_model(model_fixed)

In [8]:
wfa[2]

array([[ 0.48119032+0.j        ,  0.80505946-0.06370792j,
         0.33915892-0.0353223j ],
       [ 0.77558258+0.j        , -0.54837962-0.05902583j,
         0.21322754+0.22091043j],
       [-0.40856766+0.j        , -0.09282883-0.1870805j ,
         0.80421332+0.3777527j ]])

In [9]:
mesh = Mesh(["k", "l"], axis_names=["kx", "lmbda"])

In [10]:
mesh.build_grid(
    shape=(31, 21),
    gamma_centered=True,
    lambda_start=0.0,
    lambda_stop=1.0,
    lambda_endpoints=True,
)

In [11]:
mesh.loop(
    axis_idx=1, component_idx=1, closed=True
)  # form the lambda axis into a closed loop
print(mesh)

Mesh Summary
Type: grid
Dimensionality: 1 k-dim(s) + 1 λ-dim(s)
Number of mesh points: 651
Full shape: (31, 21, 2)
k-axes: [Axis(type=k, name=kx, size=31)]
λ-axes: [Axis(type=l, name=lmbda, size=21)]
Is a torus in k-space (all k-axes wind BZ): yes
Loops: (axis 0, comp 0, winds_bz=yes, closed=no), (axis 1, comp 1, winds_bz=no, closed=yes)


In [12]:
wfa = WFArray(lattice, mesh)

In [13]:
wfa.solve_model(model=model)

In [14]:
np.allclose(wfa[0, 0], wfa[0, -1])  # first k-point, first and last lambda points

True